In [28]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from helpers import util
from helpers import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

In [29]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
sensors_str = secrets.get_secret("SENSORS_JSON").value
sensors_data = json.loads(sensors_str)

today = datetime.date.today()

2025-11-10 13:12:35,633 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-10 13:12:35,649 INFO: Initializing external client
2025-11-10 13:12:35,650 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-10 13:12:37,187 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1272018


In [30]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

In [ ]:
import requests
import pandas as pd

start_date = today - datetime.timedelta(days=3)

last_3_days_aq_query = (
    air_quality_fg.select(["city", "pm25", "date"]).filter((air_quality_fg.date >= str(start_date)) & (air_quality_fg.date <= str(today)))
)

aq_3_days_history_df = last_3_days_aq_query.read()

aq_dfs = []

for location in sensors_data:
    aqicn_url = location['aqicn_url']
    country = location['country']
    city = location['city']
    street = location['street']
    aq_location_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
    for lagged_day in [1,2,3]:
        lag_date = today - pd.Timedelta(days=lagged_day)
        mask = (aq_3_days_history_df['city'] == city) \
            & (aq_3_days_history_df['date'].dt.date == lag_date)
        lag_value = aq_3_days_history_df.loc[mask, 'pm25']
        if not lag_value.empty:
            aq_location_today_df[f'lagged_{lagged_day}'] = lag_value.values[0]
        else:
            aq_location_today_df[f'lagged_{lagged_day}'] = None

    aq_dfs.append(aq_location_today_df)

aq_today_df = pd.concat(aq_dfs, ignore_index=True) 

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.08s) 


AttributeError: 'datetime.date' object has no attribute 'date'

In [93]:
ct = 'jonsbyn'
print(today - pd.Timedelta(days=1))
aq_3_days_history_df[(aq_3_days_history_df['city'] == ct) \
& (aq_3_days_history_df['date'].dt.date == (today - pd.Timedelta(days=1)))] \
.head()

2025-11-09


,city,pm25,date
2,jonsbyn,3.0,2025-11-09 00:00:00+00:00


In [53]:
weather_dfs = []

for location in sensors_data:
    city = location['city']
    latitude = location['latitude']
    longitude = location['longitude']
    hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
    hourly_df = hourly_df.set_index('date')

    # We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
    # We only want the daily weather data, so only get weather at 12:00
    daily_location_df = hourly_df.between_time('11:59', '12:01')
    daily_location_df = daily_location_df.reset_index()
    daily_location_df['date'] = pd.to_datetime(daily_location_df['date']).dt.date
    daily_location_df['date'] = pd.to_datetime(daily_location_df['date'])
    daily_location_df['city'] = city
    daily_location_df.loc[
        daily_location_df['date'] == pd.to_datetime('2025-11-14'),
        'precipitation_sum'
    ] = 777
    weather_dfs.append(daily_location_df)

daily_df = pd.concat(weather_dfs, ignore_index=True) 

Coordinates 59.5°N 13.5°E
Elevation 55.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 59.75°N 13.75°E
Elevation 147.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 59.5°N 12.25°E
Elevation 223.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 59.5°N 14.25°E
Elevation 146.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [54]:
daily_df.info()
daily_df.head(500)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         28 non-null     datetime64[ns]
 1   temperature_2m_mean          28 non-null     float32       
 2   precipitation_sum            28 non-null     float32       
 3   wind_speed_10m_max           28 non-null     float32       
 4   wind_direction_10m_dominant  28 non-null     float32       
 5   city                         28 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 1.0+ KB


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-10,7.05,0.0,2.880000,360.000000,skoghall
1,2025-11-11,8.25,0.1,10.464798,183.945114,skoghall
2,2025-11-12,8.50,1.6,26.717722,194.036270,skoghall
3,2025-11-13,6.25,0.0,17.555307,298.141510,skoghall
4,2025-11-14,3.15,777.0,21.889065,306.304596,skoghall
5,2025-11-15,3.60,0.0,8.287822,267.510498,skoghall
6,2025-11-16,5.20,0.0,12.313893,285.255157,skoghall
7,2025-11-10,6.45,0.0,5.760000,90.000000,jonsbyn
8,2025-11-11,7.00,0.0,8.647496,182.385895,jonsbyn
9,2025-11-12,7.75,2.2,21.252199,181.941437,jonsbyn


In [55]:
# Insert new data
air_quality_fg.insert(aq_today_df)
weather_fg.insert(daily_df, wait=True)

2025-11-10 13:49:46,063 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1272018/fs/1265761/fg/1668534


Uploading Dataframe: 100.00% |██████████| Rows 4/4 | Elapsed Time: 00:01 | Remaining Time: 00:00


2025-11-10 13:49:54,991 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1272018/fs/1265761/fg/1668535


Uploading Dataframe: 100.00% |██████████| Rows 28/28 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1272018/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-10 13:50:12,251 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-11-10 13:50:15,448 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-10 13:50:18,635 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-10 13:52:00,827 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-10 13:52:00,980 INFO: Waiting for log aggregation to finish.
2025-11-10 13:52:13,023 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 739354
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 28,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-10T12:49:54.000991Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column

In [56]:
df = weather_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.44s) 


In [61]:
df.sort_values(by="date", ascending=False).head(40)

,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
6587,2025-11-16 00:00:00+00:00,4.200000,0.0,12.594856,300.963684,jonsbyn
6585,2025-11-16 00:00:00+00:00,4.150000,0.0,12.144331,281.976105,årjäng
6584,2025-11-16 00:00:00+00:00,5.200000,0.0,12.313893,285.255157,skoghall
6586,2025-11-16 00:00:00+00:00,4.200000,0.0,12.727921,298.739685,nykroppa
6560,2025-11-15 00:00:00+00:00,3.600000,0.0,8.287822,267.510498,skoghall
6562,2025-11-15 00:00:00+00:00,2.700000,0.0,7.993298,262.234924,jonsbyn
6563,2025-11-15 00:00:00+00:00,2.700000,0.0,9.885262,236.888641,årjäng
6565,2025-11-15 00:00:00+00:00,2.250000,0.0,9.255571,256.504242,nykroppa
6571,2025-11-14 00:00:00+00:00,2.400000,777.0,18.775303,302.471161,årjäng
6581,2025-11-14 00:00:00+00:00,1.950000,777.0,20.683559,301.476776,jonsbyn
